In [1]:
import os
import time
import torch
import torchtuples as tt
import numpy as np
import easydict
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold
import os
import time
import torch
import torchtuples as tt
import numpy as np
import easydict
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm, trange
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MinMaxScaler
import torch
from torch.utils.data import DataLoader
from expected_gradient_multi_task import AttributionPriorExplainer
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import grad
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('Dataset/processdata.csv', encoding='latin-1')
data = df

date_columns = ['Date.of.Last.Contact', 'Date.of.Diagnostic']
data[date_columns] = data[date_columns].apply(pd.to_datetime, errors='coerce')
has_na = data[date_columns].isna().any(axis=1)

# My comment: why not death time - contact time?
if has_na.any():
    data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days
else:
    data['Survival_Time'] = (data['Date.of.Last.Contact'] - data['Date.of.Diagnostic']).dt.days
    
data.loc[:, 'Survival_Time'] = data['Survival_Time'].replace({-1:0})

data['indicater'] = np.where(data['Date.of.Death'].isna(), 0, 1)
columns_to_drop = ['Date.of.Death', 'Date.of.Last.Contact', 'Date.of.Diagnostic']


data.drop(columns=columns_to_drop, inplace=True)




columns_to_one_hot = ['RCBP.Name', 'Raca.Color', 'State.Civil', 'Code.Profession', 'Name.Occupation', 'Status.Address',
                      'City.Address', 'Description.of.Topography', 'Topography.Code', 'Morphology.Description',
                      'Code.of.Morphology', 'Description.of.Disease', 'Illness.Code', 'Diagnostic.means', 'Extension',
                      'Type.of.Death']

# Replace other values that are not in top 9, into "other"
for column in columns_to_one_hot:
    top_9_values = data[column].value_counts().nlargest(9).index
    data[column] = data[column].where(data[column].isin(top_9_values), 'other')

data = pd.get_dummies(data, columns=columns_to_one_hot)

columns_to_binarize = ['Gender', 'Indicator.of.Rare.Case']

lb = LabelBinarizer()
for column in columns_to_binarize:
    data[column] = lb.fit_transform(data[column])

scaler = MinMaxScaler()
data['Age'] = scaler.fit_transform(data[['Age']])



X = data.drop(['Survival_Time', 'indicater'], axis=1)
time_all = data['Survival_Time'].values
event_all = data['indicater'].values
max_time = data['Survival_Time'].max()
print(max_time)

data.head()


# Take the maximum time and divide it into any number of intervals
Tmax = 7500
num_intervals = 7

intervals = [(i * (Tmax // num_intervals), (i + 1) * (Tmax // num_intervals)) for i in range(num_intervals)]

Y = np.zeros((len(time_all), num_intervals), dtype=np.int_)

# Until the event happens, value is 1. after that, it is 0
for i, time_val in enumerate(time_all):
    for j, (left, right) in enumerate(intervals):
        if time_val > right or (left < time_val <= right):
            Y[i, j] = 1

Y = torch.Tensor(Y)

print(Y.shape)

# Creat mask matrix
W = np.zeros((len(time_all), num_intervals), dtype=np.int_)

# Until the time we know he was alive the value is 1, after that is 0
for i, (time_val, event_val) in enumerate(zip(time_all, event_all)):
    for j, (left, right) in enumerate(intervals):
        if event_val == 0 and time_val < left:
            W[i, j] = 0
        else:
            W[i, j] = 1

W = torch.Tensor(W)



for col in X.columns:
    if X[col].dtype == bool:
        X[col] = X[col].astype(int)
X_use = torch.tensor(X.values, dtype=torch.float32)
X_use.shape

# Convert n*m data into m * n * 1 
Y_transform = [Y[:, i:i+1] for i in range(Y.size(1))]
print(Y_transform[0].shape)
W_transform = [W[:, i:i+1] for i in range(W.size(1))]
print(W_transform[0].shape)


# Jingyan adds the dataloader
from torch.utils.data import Dataset, DataLoader, random_split

class MultiTaskDataset(Dataset):
    def __init__(self, data, targets, masks, event_all):
        self.data = data
        self.targets = targets
        self.masks = masks
        self.event_all = event_all

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], [self.targets[i][idx] for i in range(len(self.targets))], [self.masks[i][idx] for i in range(len(self.masks))], self.event_all[idx]



    

7408
torch.Size([14755, 7])
torch.Size([14755, 1])
torch.Size([14755, 1])


In [3]:
#Xinyu

args = easydict.EasyDict({
    "batch_size": 64,
    "cuda": True, # should set it to be true when using gpu, otherwise data would be on two devices
    "lr": 0.05,
    "seed": 1111,
    "reduce_rate": 0.95,
    "epochs": 200,
    "clip": 5.0,
    "log_interval":10,
})

full_dataset = MultiTaskDataset(X_use, Y_transform, W_transform, event_all)

# train_size = int(0.8 * len(full_dataset))
# test_size = len(full_dataset) - train_size
# train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
train_indices, test_indices = train_test_split(
    np.arange(len(Y)), stratify=event_all, random_state=1
)
train_dataset, test_dataset = (torch.utils.data.Subset(full_dataset, train_indices), 
                               torch.utils.data.Subset(full_dataset, test_indices))
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, drop_last=True)
for x, y, w, e in train_loader:
    print(x.shape)
    print(len(y), y[0].shape)
    print(len(w), w[0].shape)
    print(e.shape)
    in_features = x.shape[1]
    out_features = len(y)
    break
    

torch.Size([64, 138])
7 torch.Size([64, 1])
7 torch.Size([64, 1])
torch.Size([64])


In [4]:
import torch.nn as nn
import torch.nn.functional as F

# Made by Xinyu and modified by Dr.Li and Jingyan
class MultiTaskModel(nn.Module):
    def __init__(self, in_features, out_features):
        super(MultiTaskModel, self).__init__()
        self.num_tasks = out_features
        self.input_features = in_features

        self.shared_layers = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1),

            nn.Linear(128, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1),

            nn.Linear(128, 128),
            nn.ReLU(),
            nn.AlphaDropout(0.1)
        )
        # A Classic Multi Task Learning Framework
        self.task_layers = nn.ModuleList([nn.Linear(128, 1) for _ in range(out_features)])#after your papaer I decided to use MTL to solve this prob

    # This forward propagation logic defines the chain propagation of our idea
    def forward(self, x):
        shared_output = self.shared_layers(x)
        task_outputs = []
        for i, task_layer in enumerate(self.task_layers):
            if i == 0:
                task_output = torch.sigmoid(task_layer(shared_output))
            else:
                task_output = torch.sigmoid(task_layer(shared_output)) * task_outputs[-1]
            task_outputs.append(task_output)

        return task_outputs# Here I difined the S(x)

    # Modified by Jingyan
    def custom_loss(self, task_outputs, targets, masks):
        loss = 0
        for i, task_output in enumerate(task_outputs):
            task_target = targets[i]
            task_mask = masks[i]
            task_loss = F.binary_cross_entropy(task_output, task_target.float(), reduction='none')
            task_loss = task_loss * task_mask.float() # [2048, 1]
            # print('task_loss', task_loss.shape)
            loss += task_loss.sum() / task_mask.sum()
        return loss



In [5]:
import torch
from torch.utils.data import DataLoader
from expected_gradient_multi_task import AttributionPriorExplainer
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import grad


# Define device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiTaskModel(in_features, out_features).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
lambda_reg = 0.01
APExp = AttributionPriorExplainer(train_dataset,args.batch_size)


# Early stopping and checkpointing variables
best_val_loss = float('inf')
epochs_no_improve = 0
early_stop_patience = 25
checkpoint_path = 'outputfiles/best_model.pth'

def save_checkpoint(state, is_best, filename='outputfiles/checkpoint.pth'):
    torch.save(state, filename)
    if is_best:
        torch.save(state, checkpoint_path)

def compute_gradients(features, task_output):
        inp_grad = grad(outputs=torch.log(task_output),
                        inputs=features,
                        grad_outputs=torch.ones_like(task_output),
                        create_graph=True)[0]
        return inp_grad        

for epoch in trange(args.epochs):
    model.train()
    running_loss = 0.0
    gradient_norms = []
    for batch_idx, (X_train, targets, masks, event_train) in enumerate(train_loader):
        
        X_train = X_train.to(device)
        X_train.requires_grad = True
        targets = [target.to(device) for target in targets]
        masks = [mask.to(device) for mask in masks]
        optimizer.zero_grad()
        task_outputs = model(X_train)
        
        ## get Expected Gradients attribution
        eg = APExp.shap_values(model,X_train)

        # Base loss computation
        L_base = model.custom_loss(task_outputs, targets, masks)

        Omega_smooth = 0
        prev_attributions = None
        for t in range(out_features):
            current_attributions = eg[t] #compute_gradients(X_train, task_outputs[t])#eg[t] # compute_gradients(X_train, task_outputs[t])
            # Compute smoothing regularization
            if prev_attributions is not None:
                diff = current_attributions - prev_attributions
                Omega_smooth += torch.abs(diff)              
            prev_attributions = current_attributions

        # The formula in project description is summation, but I got better result by taking average.
        Omega_smooth = torch.sum(Omega_smooth)  # Omega_smooth.mean() #torch.sum(Omega_smooth) 
        loss = L_base + (lambda_reg * Omega_smooth)
        loss.backward(retain_graph=True)
         # Compute and store gradient norms
        total_norm = 0
        for param in model.parameters():
            param_norm = param.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
        total_norm = total_norm ** (1. / 2)
        gradient_norms.append(total_norm)
        
        
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=args.clip)
        optimizer.step()
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    avg_grad_norm = sum(gradient_norms) / len(gradient_norms)
    # print(f'End of Epoch {epoch}, Average Training Loss: {avg_loss:.4f}')
    print(f'End of Epoch {epoch}, Average Training Loss: {avg_loss:.4f}, Average Gradient Norm: {avg_grad_norm:.4f}')

    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val, val_targets, val_masks, event_val in test_loader:
            X_val = X_val.to(device)
            val_targets = [target.to(device) for target in val_targets]
            val_masks = [mask.to(device) for mask in val_masks]
            val_outputs = model(X_val)
            loss = model.custom_loss(val_outputs, val_targets, val_masks)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(test_loader)
    print(f'End of Epoch {epoch}, Average Validation Loss: {avg_val_loss:.4f}')
    
    # Update the learning rate based on the validation loss
    scheduler.step(avg_val_loss)
    
    # Print the current learning rate
    current_lr = scheduler.optimizer.param_groups[0]['lr']
    print(f'Current Learning Rate: {current_lr:.6f}')

    # Check if this is the best validation loss we've seen so far
    is_best = avg_val_loss < best_val_loss
    if is_best:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_val_loss': best_val_loss,
        'optimizer': optimizer.state_dict(),
    }, is_best)
    
    # Early stopping
    if epochs_no_improve >= early_stop_patience:
        print(f'Early stopping triggered after {epoch+1} epochs.')
        break




  0%|          | 1/200 [00:04<15:19,  4.62s/it]

End of Epoch 0, Average Training Loss: 2.0786, Average Gradient Norm: 2.6651
End of Epoch 0, Average Validation Loss: 1.2684
Current Learning Rate: 0.010000


  1%|          | 2/200 [00:07<12:23,  3.76s/it]

End of Epoch 1, Average Training Loss: 1.7817, Average Gradient Norm: 3.5047
End of Epoch 1, Average Validation Loss: 1.3096
Current Learning Rate: 0.010000
End of Epoch 2, Average Training Loss: 1.6873, Average Gradient Norm: 3.3423


  2%|▏         | 3/200 [00:10<11:27,  3.49s/it]

...

End of Epoch 40, Average Training Loss: 1.4299, Average Gradient Norm: 5.8032
End of Epoch 40, Average Validation Loss: 1.0652
Current Learning Rate: 0.000001


 21%|██        | 42/200 [03:15<09:10,  3.49s/it]

End of Epoch 41, Average Training Loss: 1.4518, Average Gradient Norm: 5.7269
End of Epoch 41, Average Validation Loss: 1.0653
Current Learning Rate: 0.000000


 21%|██        | 42/200 [03:18<12:27,  4.73s/it]

End of Epoch 42, Average Training Loss: 1.4390, Average Gradient Norm: 6.1429
End of Epoch 42, Average Validation Loss: 1.0653
Current Learning Rate: 0.000000
Early stopping triggered after 43 epochs.


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiTaskModel(in_features, out_features).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Load the checkpoint
checkpoint = torch.load('outputfiles/best_model.pth')

# Restore the model and optimizer state
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])

# Optionally, restore other variables such as epoch and best validation loss
start_epoch = checkpoint['epoch']
best_val_loss = checkpoint['best_val_loss']

print(f"Loaded model from checkpoint at epoch {start_epoch} with best validation loss {best_val_loss:.4f}")

Loaded model from checkpoint at epoch 18 with best validation loss 1.0553


In [7]:
class Cindex(torch.nn.Module):
    def __init__(self):
        super(Cindex, self).__init__()

    def forward(self, y, y_hat, status):
        if not torch.is_tensor(y):
            y = torch.Tensor(y).to("cuda")
        if not torch.is_tensor(y_hat):
            y_hat = torch.Tensor(y_hat).to("cuda")
        if not torch.is_tensor(status):
          status = torch.Tensor(status).to("cuda")
            
        N = y.size(0)
        total_pairs = 0
        c = 0
        
        for i in trange(N):
            a = y[i]
            a_hat = y_hat[i]
            b = y[i+1:]
            b_hat = y_hat[i+1:]
            astatus = status[i]
            bstatus = status[i+1:]
            
            a_greater = a >= b
            a_hat_greater = a_hat >= b_hat
            bstatus_1 = bstatus == 1
            
            a_lesser =  a <= b
            a_hat_lesser = a_hat <= b_hat
            astatus_1 = astatus == 1
            c1 = torch.logical_and(a_greater.to("cuda"), a_hat_greater.to("cuda"))
            c1 = torch.logical_and(c1, bstatus_1.to("cuda"))
            
            c2 = torch.logical_and(a_lesser.to("cuda"), a_hat_lesser.to("cuda"))
            c2 = torch.logical_and(c2, astatus_1.to("cuda"))
            cc = torch.logical_or(c1, c2)
            c += torch.sum(cc.long())
            
            tp1 = torch.logical_and(a_lesser.to("cuda"), astatus_1.to("cuda"))
            tp2 = torch.logical_and(a_greater.to("cuda"), bstatus_1.to("cuda"))
            tp = torch.logical_or(tp1, tp2)
            total_pairs += torch.sum(tp.long())
            
        
        # for i in trange(N):
        #     for j in range(i + 1, N):
        #         a = y[i]
        #         b = y[j]
        #         a_hat = y_hat[i]
        #         b_hat = y_hat[j]
        #         astatus = status[i]
        #         bstatus = status[j]
        #         if (a >= b and a_hat >= b_hat and bstatus == 1) or (a <= b and a_hat <= b_hat and astatus == 1):
        #             c += 1
        #         if (a <= b and astatus==1) or (b <= a and bstatus == 1):
        #             total_pairs += 1
        
        print(c)
        print(total_pairs)
        outcome = c / total_pairs
        return outcome.cpu().item()


# made by Xinyu
def binarize_and_sum_columns(output_list):
    def binarize_list(input_list):
        tensor = torch.Tensor(input_list)
        # print(input_list.max() == input_list.min())
        binary_tensor = (tensor >= 0.5).float()
        return binary_tensor

    result = binarize_list(output_list[0])
    for i in range(1, len(output_list)):
        binary_column = binarize_list(output_list[i])
        # print(binary_column.max() == binary_column.min())
        result += binary_column

    return result

# Created by Jingyan
model.eval()
test_predictions = []

# Disable gradient calculations
with torch.no_grad():
    for X_test, _, _, _ in test_loader:
        # Forward pass
        X_test = X_test.to(device)
        task_outputs_ = model(X_test)

        # Store the predictions
        test_predictions.append(task_outputs_)
test_predictions = [torch.cat([preds[i] for preds in test_predictions]) for i in range(len(test_predictions[0]))]
Y_hat_test = binarize_and_sum_columns(test_predictions)
Y_hat_test = Y_hat_test.squeeze()
print(Y_hat_test.unique())



test_trues = []
test_statuses = []
for _, test_targets, test_masks, test_status in test_loader:
    true_label = [test_targets[i]*test_masks[i] for i in range(len(test_targets))]
    test_trues.append(true_label)
    test_statuses.append(test_status)

test_trues = [torch.cat([preds[i] for preds in test_trues]) for i in range(len(test_trues[0]))]
test_statuses = torch.cat([status for status in test_statuses])

print(len(test_trues))
print(test_trues[0].shape)
print(test_statuses.shape)

Y_true_test = binarize_and_sum_columns(test_trues)
Y_true_test = Y_true_test.squeeze()
print(Y_hat_test.unique())
cindex_calculator = Cindex()
c11_test = cindex_calculator(Y_true_test, Y_hat_test, test_statuses)
print(c11_test)


tensor([0., 1., 2., 3., 4., 7.], device='cuda:0')
7
torch.Size([3648, 1])
torch.Size([3648])
tensor([0., 1., 2., 3., 4., 7.], device='cuda:0')


100%|██████████| 3648/3648 [00:01<00:00, 2381.20it/s]

tensor(5437393, device='cuda:0')
tensor(5569534, device='cuda:0')
0.976274311542511


In [8]:
from sksurv.metrics import integrated_brier_score

dtype = [('cens', bool), ('time', int)]
# Create the structured array
all_array = np.array(list(zip(event_all.astype(bool), Y.sum(axis=1).cpu())), dtype=dtype)
test_array = np.array(list(zip(test_statuses.cpu().numpy().astype(bool), Y_true_test.cpu())), dtype=dtype)
_times = np.arange(1, 7)
probs = np.column_stack([item.cpu().numpy() for item in test_predictions])
integrated_brier_score(all_array, test_array, probs[:, 1:7], _times)

0.06945544592747543

In [ ]:
# 0.9685770869255066 0.9811781644821167, 0.06966835282499209 0.07313522417625694
# 0.9815456867218018, 0.13459566972554354
# 0.9756746292114258, 0.06753334177025772

In [ ]:
# C-index

# batch 64 and lambda 0.1 = 0.9747766101661264
# batch 64 and lambda 0.01 = 0.9727702104237926
# batch 64 and lambda 0.001 = 0.9609226584434509
# batch 32 and lambda 0.001 = 0.9801711440086365
# Default batch 64 = 0.980457603931427


# batch 64 and lambda 0.001 = 0.9748636484146118 # method EG  0.9782829284667969
# batch 64 and lambda 0.01 = 0.9813091158866882 # method gradient
# batch 64  = 0.9793468713760376  # method No regularization


In [ ]:
# IB Score

# batch 64 and lambda 0.001 = 0.06530525413905933
# Default batch 64 = 0.06640703696079571


# batch 64 and lambda 0.01 = 0.06833509861869876 # method EG  0.06989871045566962
# batch 64 and lambda 0.01 = 0.13501370015201017 # method gradient
# batch 64 0.06639405091289838 # method No regularization


In [29]:
from sklearn.metrics import classification_report

print("Default model")
print(classification_report(Y_true_test.cpu(), Y_hat_test.cpu()))

Default model
              precision    recall  f1-score   support

         0.0       1.00      0.85      0.92      1026
         1.0       0.81      0.63      0.71      1009
         2.0       0.28      0.36      0.31       404
         3.0       0.19      0.11      0.14       507
         4.0       0.25      0.09      0.13       406
         5.0       0.00      0.00      0.00       214
         6.0       0.00      0.00      0.00        64
         7.0       0.02      0.89      0.03        18

    accuracy                           0.48      3648
   macro avg       0.32      0.37      0.28      3648
weighted avg       0.59      0.48      0.52      3648


C:\Users\Atlas\anaconda3\envs\future\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Atlas\anaconda3\envs\future\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Atlas\anaconda3\envs\future\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
from sklearn.metrics import classification_report

print("Reg model")
print(classification_report(Y_true_test.cpu(), Y_hat_test.cpu()))

Reg model
              precision    recall  f1-score   support

         0.0       0.97      0.85      0.91      1026
         1.0       0.73      0.70      0.71      1009
         2.0       0.26      0.17      0.20       404
         3.0       0.24      0.19      0.21       507
         4.0       0.60      0.01      0.01       406
         5.0       0.00      0.00      0.00       214
         6.0       0.00      0.00      0.00        64
         7.0       0.01      0.89      0.03        18

    accuracy                           0.48      3648
   macro avg       0.35      0.35      0.26      3648
weighted avg       0.60      0.48      0.51      3648


C:\Users\Atlas\anaconda3\envs\future\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Atlas\anaconda3\envs\future\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Atlas\anaconda3\envs\future\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
Y_true_test

tensor([1., 0., 4.,  ..., 0., 2., 5.])

In [40]:
Y_hat_test

tensor([1., 0., 7.,  ..., 0., 1., 7.], device='cuda:0')

In [11]:
train_trues = []
train_statuses = []
for _, train_targets, train_masks, train_status in train_loader:
    true_label = [train_targets[i]*train_masks[i] for i in range(len(train_targets))]
    train_trues.append(true_label)
    train_statuses.append(train_status)
train_trues = [torch.cat([preds[i] for preds in train_trues]) for i in range(len(train_trues[0]))]
train_statuses = torch.cat([status for status in train_statuses])

print(len(train_trues))
print(train_trues[0].shape)
print(train_statuses.shape)

Y_true_train = binarize_and_sum_columns(train_trues)
Y_true_train = Y_true_train.squeeze()
print(Y_true_train.shape)

7
torch.Size([11008, 1])
torch.Size([11008])
torch.Size([11008])


In [12]:
def unique_value_counts(tensor):
    # Convert the PyTorch tensor to a NumPy array
    np_array = tensor.cpu().numpy()
    
    # Use np.unique to get unique values and their counts
    unique_values, counts = np.unique(np_array, return_counts=True)
    
    # Convert the results back to PyTorch tensors (if needed)
    unique_values_tensor = torch.from_numpy(unique_values)
    counts_tensor = torch.from_numpy(counts)
    
    return unique_values_tensor, counts_tensor

# Example usage
unique_values, counts = unique_value_counts(Y_true_train)

print("Unique Values:", unique_values)
print("Counts:", counts)

Unique Values: tensor([0., 1., 2., 3., 4., 5., 6., 7.])
Counts: tensor([3065, 3019, 1206, 1598, 1211,  675,  191,   43])


In [42]:
unique_values, counts = unique_value_counts(Y_true_test)

print("Unique Values:", unique_values)
print("Counts:", counts)

Unique Values: tensor([0., 1., 2., 3., 4., 5., 6., 7.])
Counts: tensor([1026, 1009,  404,  507,  406,  214,   64,   18])


In [33]:

unique_values, counts = unique_value_counts(Y_hat_test)

print("Unique Values:", unique_values)
print("Counts:", counts)

Unique Values: tensor([0., 1., 2., 3., 4., 5., 7.])
Counts: tensor([ 874,  778,  527,  283,  142,    4, 1040])


In [13]:
unique_values, counts = unique_value_counts(Y_hat_test)

print("Unique Values:", unique_values)
print("Counts:", counts)

Unique Values: tensor([0., 1., 2., 3., 4., 7.])
Counts: tensor([ 903,  963,  255,  401,    5, 1121])


0.06640703696079571

In [141]:
test_array['time'].min()

0

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [9.99568641e-01, 4.11797166e-01, 1.86812818e-01, ...,
        8.14564526e-03, 8.70392032e-05, 1.07667552e-06],
       [9.99903202e-01, 1.09691195e-01, 4.27960083e-02, ...,
        4.37506149e-03, 2.17447068e-05, 7.80379014e-06],
       ...,
       [9.99997616e-01, 9.91505980e-01, 9.46225166e-01, ...,
        4.11568075e-01, 5.07023670e-02, 1.37454060e-06],
       [9.99976754e-01, 7.93147922e-01, 3.54994208e-01, ...,
        1.37929770e-03, 1.00549541e-05, 1.67736189e-08],
       [4.05517123e-17, 2.13318832e-27, 2.10200944e-27, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]], dtype=float32)

In [120]:
test_predictions[0][:5]

tensor([[1.0000],
        [0.9996],
        [0.9999],
        [0.9992],
        [0.9994]], device='cuda:0')